In [31]:
import numpy as np
    

class Matrix(object):
    '''
    Класс, представляющий собой матрицу и многие методы к ней
    '''
    
    def __init__(self, matr):
        types = (np.ndarray, list, tuple)
        
        if type(matr) not in types:
            raise TypeError('Не могу обработать такой тип')
            
        if type(matr[0]) not in types:
            raise ValueError('Объект не двумерный')
        
        if type(matr[0][0]) in types:
            raise ValueError('Слишком много измерений')
        
        a = len(matr[0])
        for row in matr:
            if type(row) not in types:
                raise ValueError('Матрица не целостная')
            
            if len(row) != a:
                raise ValueError('Объект не прямоугольной формы')
            a = len(row)
            
            for item in row:
                if type(item) in types:
                    raise ValueError('Где-то лишнее измерение')
            
        self.matrix = np.array(matr, dtype='float')
        
    def add_row(self, row):
        # Добавление строки в матрицу
        if len(row) != self.matrix.shape[1]:
            raise TypeError('Длинна строки не совпадает с длинной матрицы!')
            
        self.matrix = np.vstack([self.matrix, row])
        
        return Matrix(self.matrix)
        
    def add_column(self, column):
        # Добавление столбца в матрицу
        if len(column) != self.matrix.shape[0]:
            raise TypeError('Высота столбца не совпадает с высотой матрицы!')
            
        self.matrix = np.hstack([self.matrix, np.reshape(column, (len(column), 1))])
        
        return Matrix(self.matrix)
    
    @property
    def shape(self):
        # Функция для получения формы матрицы
        return self.matrix.shape
    
    def __matmul__(self, other):
        # Математическое умножение матриц
        if self.matrix.shape[0] != other.matrix.shape[1]:
            raise ValueError('Высота первой матрицы не совпадает с шириной второй!')
        
        new_M = np.zeros((self.matrix.shape[0], other.matrix.shape[1]))
        for i in range(self.matrix.shape[0]):
            for j in range(other.matrix.shape[1]):
                new_M[i ,j] = sum(self.matrix[i, :] * other.matrix[:, j])
                
        return Matrix(new_M)
    
    def __mul__(self, other):
        # Умножение матрицы на число или методом Адамара
        if type(other) == Matrix:
            return Matrix(self.matrix * other.matrix) 
        else:
            return Matrix(self.matrix * other)
    
    def __add__(self, other):
        # Сложение матриц
        if self.matrix.shape != other.matrix.shape:
            raise TypeeError('Матрицы разной размерности!')
        new_M = np.zeros(self.matrix.shape)
        
        for i in range(self.matrix.shape[0]):
            for j in range(self.matrix.shape[1]):
                new_M[i, j] = self.matrix[i, j] + other.matrix[i, j]
        
        return Matrix(new_M)
        
    def __sub__(self, other):
        # Вычетание матриц
        if self.matrix.shape != other.matrix.shape:
            raise TypeeError('Матрицы разной размерности!')
        new_M = np.zeros(self.matrix.shape)
        
        for i in range(self.matrix.shape[0]):
            for j in range(self.matrix.shape[1]):
                new_M[i, j] = self.matrix[i, j] - other.matrix[i, j]
        
        return Matrix(new_M)
    
    @property
    def T(self):
        # Транспонирование матрицы
        TM = np.zeros(self.matrix.shape[::-1])
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                TM[j, i] = self.matrix[i, j]
                
        return Matrix(TM)
    
    @property
    def trace(self):
        # След матрицы
        return sum(self.m_diag)
    
    def kron(self, other):
        # Умножение матриц методом Кронекера
        rows = []
        for i in range(len(self.matrix)):
            row = []
            for j in range(len(self.matrix[0])): 
                row.append(other.matrix * self.matrix[i, j]) 
            rows.append(row)
        
        n_rows = []
        for row in rows:
            n_row = row[0]
            for item in row[1:]:
                n_row = np.hstack([n_row, item])
            n_rows.append(n_row)
            
        new_M = n_rows[0]
        for row in n_rows[1:]:
            new_M = np.vstack([new_M, row])        
        
        return Matrix(new_M)
    
    @property
    def m_diag(self):
        # Получение главной диагонали матрицы
        tr = []
        for i in range(min(self.matrix.shape)):
            tr.append(self.matrix[i, i])
        return tr
    
    def __pow__(self, other):
        # Возведение матрицы в степень
        if other < 0:
            raise ValueError('Степень должна быть неотрицательной!')
        
        if other == 0:
            return Matrix(np.diag(np.ones(min(self.shape))))
        
        new_M = Matrix(self.matrix)
        for i in range(other - 1):
            new_M = new_M @ Matrix(self.matrix)
            
        return new_M
    
    @property
    def inv(self):
        # Обратная матрица
        if self.det == 0:
            raise ValueError('Определитель не должен быть равен нулю!')
            
        return Matrix(np.linalg.inv(self.matrix))
    
        # Функция добавлена для полноты функционала, реализована через numpy
        
    @property
    def det(self):
        # Определитель матрицы 
        if self.matrix.shape[0] != self.matrix.shape[1]:
            raise TypeError('Матрица должна быть квадратной!')
            
        return np.linalg.det(self.matrix)
    
        # Функция добавлена для полноты функционала, реализована через numpy
        
    def __repr__(self):
        # Функция, отвечающая за вывод данных о матрице 
        return str(self.matrix)

In [14]:
# Пример создания матрицы

a = Matrix([[1, 2], [3, 4]])
a, type(a)

([[1. 2.]
  [3. 4.]],
 __main__.Matrix)

In [15]:
# Пример умножения матриц

a = Matrix([[1, 2], [3, 4]])
b = Matrix([[4, 3], [2, 1]])
a @ b

[[ 8.  5.]
 [20. 13.]]

In [16]:
# Пример произведения Адамара

a = Matrix([[1, 2], [3, 4]])
b = Matrix([[4, 3], [2, 1]])
a * b

[[4. 6.]
 [6. 4.]]

In [17]:
# Пример умножения матрицы на число

a = Matrix([[1, 2], [3, 4]])
a * 2

[[2. 4.]
 [6. 8.]]

In [18]:
# Пример сложенния матриц

a = Matrix([[1, 2], [3, 4]])
b = Matrix([[4, 3], [2, 1]])
a + b

[[5. 5.]
 [5. 5.]]

In [19]:
# Пример вычитания матриц

a = Matrix([[1, 2], [3, 4]])
b = Matrix([[4, 3], [2, 1]])
a - b

[[-3. -1.]
 [ 1.  3.]]

In [22]:
# Пример транспонирования матрицы

a = Matrix([[1, 2], [3, 4], [5, 6]])
a.T

[[1. 3. 5.]
 [2. 4. 6.]]

In [23]:
# Пример получения главной диагонали

a = Matrix([[1, 2], [3, 4]])
a.m_diag

[1.0, 4.0]

In [24]:
# Пример получения следа матрицы

a = Matrix([[1, 2], [3, 4]])
a.trace

5.0

In [25]:
# Пример произведения Кронекера

a = Matrix([[1, 2], [3, 4]])
b = Matrix([[4, 3], [2, 1]])
a.kron(b)

[[ 4.  3.  8.  6.]
 [ 2.  1.  4.  2.]
 [12.  9. 16. 12.]
 [ 6.  3.  8.  4.]]

In [26]:
# Пример возведения матрицы в степень

a = Matrix([[1, 2], [3, 4]])
a**2

[[ 7. 10.]
 [15. 22.]]

In [29]:
# Пример добавления строки

a = Matrix([[1, 2], [3, 4]])
a.add_row([5, 6])

[[1. 2.]
 [3. 4.]
 [5. 6.]]

In [30]:
# Пример добавления столбца

a = Matrix([[1, 2], [3, 4]])
a.add_column([5, 6])

[[1. 2. 5.]
 [3. 4. 6.]]